# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
import sys
sys.path.insert(1, '../WeatherPy')
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
citiescsvdf = pd.read_csv("../WeatherPy/citiesdfOutput.csv", encoding = "UTF-8", index_col = 0)
citiescsvdf

,City Name,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime
0,Thompson,55.74,-97.86,8.60,84,90,13.87,CA,-21600
1,Mackay,-21.15,149.20,78.80,88,75,9.17,AU,36000
2,Saint George,37.10,-113.58,39.99,80,90,1.90,US,-25200
3,Rafai,4.95,23.92,69.89,53,100,1.88,CF,3600
4,Cape Town,-33.93,18.42,62.01,82,75,5.82,ZA,7200
...,...,...,...,...,...,...,...,...,...
565,Saint-Francois,46.42,3.91,42.80,70,90,10.29,FR,3600
566,Zhigansk,66.77,123.37,-10.34,88,100,19.89,RU,32400
567,Nambour,-26.63,152.95,82.99,79,100,1.99,AU,36000
568,Nizhneivkino,58.19,49.52,-10.37,90,5,2.04,RU,10800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure map
gmaps.configure(api_key = g_key)
locations = citiescsvdf[["City Latitude", "City Longitude"]].astype(float)

# Set heat weight
humidWeight = citiescsvdf["Humidity"].astype(float)
# Set initialize map with initial map position
fig = gmaps.figure(zoom_level = 2, center = (32.715, -117.161))
# Create heat map layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidWeight, dissipating = False, max_intensity=100,
                                 point_radius = 1.1)
# Add heat map layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create initial list of good cities 
goodCitiesdf = citiescsvdf.loc[(citiescsvdf["Max Temperature"] > 75) & 
                               (citiescsvdf["Max Temperature"] < 90)&
                               (citiescsvdf["Wind Speed"] < 10)&
                               (citiescsvdf["Cloud Coverage"] == 0),]
# Drop any rows without a value
goodCitiesdf = goodCitiesdf.dropna(how = 'any')
goodCitiesdf

,City Name,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime
60,Wanning,18.80,110.40,77.81,71,0,7.16,CN,28800
86,Mahibadhoo,3.78,72.97,82.29,73,0,9.57,MV,18000
162,Careiro Da Varzea,-3.20,-59.81,77.00,88,0,3.36,BR,-14400
232,Champasak,14.75,106.00,84.20,58,0,2.24,LA,25200
247,Conceicao Do Araguaia,-8.26,-49.26,75.52,74,0,2.30,BR,-10800
326,Kulhudhuffushi,6.62,73.07,81.61,71,0,8.05,MV,18000
401,Palasa,18.77,84.42,76.01,48,0,4.23,IN,19800
538,Tamandare,-8.76,-35.10,75.51,78,0,7.96,BR,-10800
557,Nakhon Phanom,17.33,104.42,84.99,54,0,4.70,TH,25200


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Add "Hotel Names" column
goodCitiesdf["Hotel Names"] = ""
# Rename dataframe to fit assignment instructions
hotel_df = goodCitiesdf
hotel_df

,City Name,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime,Hotel Names
60,Wanning,18.80,110.40,77.81,71,0,7.16,CN,28800,
86,Mahibadhoo,3.78,72.97,82.29,73,0,9.57,MV,18000,
162,Careiro Da Varzea,-3.20,-59.81,77.00,88,0,3.36,BR,-14400,
232,Champasak,14.75,106.00,84.20,58,0,2.24,LA,25200,
247,Conceicao Do Araguaia,-8.26,-49.26,75.52,74,0,2.30,BR,-10800,
326,Kulhudhuffushi,6.62,73.07,81.61,71,0,8.05,MV,18000,
401,Palasa,18.77,84.42,76.01,48,0,4.23,IN,19800,
538,Tamandare,-8.76,-35.10,75.51,78,0,7.96,BR,-10800,
557,Nakhon Phanom,17.33,104.42,84.99,54,0,4.70,TH,25200,


In [6]:
# Build Google Places request URL
baseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="

# Variable values
# Lat/Lng will be set in for loop on hotel_df below
radius = 5000
searchType = "lodging"
searchWord = "hotel"

# Building subparts
radiusURL = f'&radius={radius}'
typeURL = f'&type={searchType}'
searchURL = f'&keyword={searchWord}'
apikeyURL = f'&key={g_key}'

In [10]:
# Initialize iterators and error lists
hotelIterator = 1
hotelMissIndex = []
hotelMiss = []

for index, row in hotel_df.iterrows():
    # Create try/except to compensate for errors
    searchLat = row["City Latitude"]
    searchLon = row["City Longitude"]
    try:
        print(f'Searching for hotel in city no.{hotelIterator}, "{row["City Name"]}"')
        fullRequestURL = f'{baseURL}{searchLat},{searchLon}{radiusURL}{typeURL}{searchURL}{apikeyURL}'
        jsonObj = requests.get(fullRequestURL).json()
        #Pull data from json response into lists
        print(f'Found "{jsonObj["results"][0]["name"]}"')
        hotel_df.loc[index, "Hotel Names"] = jsonObj["results"][0]["name"]
    except:
        print(f'        ...No hotels found. Recording and resuming.')
        hotelMiss.append(row["City Name"])
        hotelMissIndex.append(index)
    hotelIterator += 1
    time.sleep(1)

Searching for hotel in city no.1, "Wanning"
Found "Meet Joy Boutique Hotel"
Searching for hotel in city no.2, "Mahibadhoo"
Found "Noovilu Suites - The luxury guest house in Maldives"
Searching for hotel in city no.3, "Careiro Da Varzea"
        ...No hotels found. Recording and resuming.
Searching for hotel in city no.4, "Champasak"
Found "Kingfisher Ecolodge"
Searching for hotel in city no.5, "Conceicao Do Araguaia"
Found "Tarumã Tropical Hotel"
Searching for hotel in city no.6, "Kulhudhuffushi"
Found "The Local Stay - NM1"
Searching for hotel in city no.7, "Palasa"
Found "Hotels at kssibugga"
Searching for hotel in city no.8, "Tamandare"
Found "Ancoradouro Hotel Tamandaré - PE"
Searching for hotel in city no.9, "Nakhon Phanom"
        ...No hotels found. Recording and resuming.


In [11]:
# Remove any locations that did not find a hotel
hotel_df.drop(hotelMissIndex, inplace = True)
hotel_df

,City Name,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime,Hotel Name,Hotel Names
60,Wanning,18.80,110.40,77.81,71,0,7.16,CN,28800,NaN,Meet Joy Boutique Hotel
86,Mahibadhoo,3.78,72.97,82.29,73,0,9.57,MV,18000,NaN,Noovilu Suites - The luxury guest house in Mal...
232,Champasak,14.75,106.00,84.20,58,0,2.24,LA,25200,NaN,Kingfisher Ecolodge
247,Conceicao Do Araguaia,-8.26,-49.26,75.52,74,0,2.30,BR,-10800,NaN,Tarumã Tropical Hotel
326,Kulhudhuffushi,6.62,73.07,81.61,71,0,8.05,MV,18000,NaN,The Local Stay - NM1
401,Palasa,18.77,84.42,76.01,48,0,4.23,IN,19800,NaN,Hotels at kssibugga
538,Tamandare,-8.76,-35.10,75.51,78,0,7.96,BR,-10800,NaN,Ancoradouro Hotel Tamandaré - PE


In [12]:
# Output dataframe to csv
hotel_df.to_csv("hotelOutput.csv", index=True, header=True)

In [13]:
# Read city hotel dataframe
#(for continuation of study without needing to re-request to the source)
hotel_df = pd.read_csv("hotelOutput.csv", encoding = "UTF-8", index_col = 0)
hotel_df

,City Name,City Latitude,City Longitude,Max Temperature,Humidity,Cloud Coverage,Wind Speed,City Country,City Datetime,Hotel Name,Hotel Names
60,Wanning,18.80,110.40,77.81,71,0,7.16,CN,28800,NaN,Meet Joy Boutique Hotel
86,Mahibadhoo,3.78,72.97,82.29,73,0,9.57,MV,18000,NaN,Noovilu Suites - The luxury guest house in Mal...
232,Champasak,14.75,106.00,84.20,58,0,2.24,LA,25200,NaN,Kingfisher Ecolodge
247,Conceicao Do Araguaia,-8.26,-49.26,75.52,74,0,2.30,BR,-10800,NaN,Tarumã Tropical Hotel
326,Kulhudhuffushi,6.62,73.07,81.61,71,0,8.05,MV,18000,NaN,The Local Stay - NM1
401,Palasa,18.77,84.42,76.01,48,0,4.23,IN,19800,NaN,Hotels at kssibugga
538,Tamandare,-8.76,-35.10,75.51,78,0,7.96,BR,-10800,NaN,Ancoradouro Hotel Tamandaré - PE


In [14]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Names}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{City Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["City Latitude", "City Longitude"]]

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    locations, 
    info_box_content = hotel_info
)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))